### Sample code - stargazers count

In [1]:
import pandas as pd
merge = pd.read_csv('data/merge_bq_repo_name.csv') # merge_bq_repo_name is the repo name I merged 
merge.shape

(16467, 2)

In [2]:
merge.head(10)

,repo_name,company_name
0,000webhost/000webhost-sdk-js,000webhost
1,100dayproject/polymer-web-example,100dayproject
2,18F/18f-cf-release,18F
3,18F/api-program,18F
4,18F/caddy-boshrelease,18F
5,18F/cf-abacus,18F
6,18F/cf-meteor-buildpack,18F
7,18F/cg-cf-release,18F
8,18F/cg-style,18F
9,18F/cg-style-gem,18F


In [3]:
merge_repo = list(merge['repo_name']) # get only repo name from dataset
len(merge_repo)

16467

### Create pickle to save data in every iteration

In [4]:
# -- save picle  
import pickle
filepath = 'stargazer_time.pkl'

def batch(time_list):
    
    with open(filepath, 'ab') as f:
        pickle.dump(time_list, f)

### Scrape data using github API

In [5]:
import requests
import time
import json


auth = ('ChuxinYang','b5b61b3346c994d7939ace05f52a3efcdd512066') 

stargazer_time = []

for j in range(len(merge_repo)):

    
    page = 1
    while True: 
        
        # use github API to scrape data
        get = requests.get('https://api.github.com/repos/'+ str(merge_repo[j]) +'/stargazers?page='+ str(page) +'&per_page=100/access_tokens',
                    headers={'Accept': 'application/vnd.github.v3.star+json'},
                      auth = auth)
        get_json = json.loads(get.text)
        time.sleep(2) # rate limit
        
        # test if this repo still exist
        if type(get_json) == dict:
            break
            
        else:
            if len(get_json) == 0: # if there is nothing inside, break out of the loop and look for next repo
                break
 
                
            for i in range(len(get_json)):

                repo_tuple = tuple([merge_repo[j]]) # save the repo name first
                repo_tuple += tuple([get_json[i]['user']['login']]) # save the stargazer name
                repo_tuple += tuple([get_json[i]['user']['id']]) # save the stargazer id
                repo_tuple += tuple([get_json[i]['starred_at']]) # save the starred at time

                batch(repo_tuple) # call batch function to save repo_tuple in a pickle file
                    
            page += 1 
    print(j) # print which repo we are scraping
    

0
1
2
3
4
5


### Open pickle file and read data

In [6]:
import pandas as pd 
import pickle
colnames = ['repo_name', 'stargazer_name','stargazer_id','starred_at_time'] # set column names

info = []
with open('stargazer_time.pkl', 'rb') as fr:
    try:
        while True:
            info.append(pickle.load(fr))
    except EOFError:
        pass
    
repo_info = pd.DataFrame(info, columns=colnames)
repo_info

,repo_name,stargazer_name,stargazer_id,starred_at_time
0,000webhost/000webhost-sdk-js,Lazymuxhost,49286156,2019-06-18T23:36:11Z
1,100dayproject/polymer-web-example,kimthangatm,7090588,2016-03-28T06:03:17Z
2,100dayproject/polymer-web-example,lephuhai,7351351,2016-03-28T06:55:19Z
3,100dayproject/polymer-web-example,trquoccuong,7078282,2016-03-30T14:09:13Z
4,18F/18f-cf-release,rogeruiz,706004,2016-12-08T18:04:25Z
5,18F/18f-cf-release,rememberlenny,1332366,2017-03-03T18:12:03Z
6,18F/api-program,leahbannon,4323988,2015-02-07T19:58:56Z
7,18F/api-program,adelevie,86790,2015-02-07T20:56:33Z
8,18F/api-program,konklone,4592,2015-02-08T05:38:35Z
9,18F/api-program,parkr,237985,2015-02-08T05:51:12Z
